In [67]:
import os,sys
import pandas as pd
%matplotlib inline
os.chdir("jeme/JEME vs Ripple/")
from sklearn.model_selection import cross_val_score, StratifiedKFold, KFold
from sklearn.ensemble import GradientBoostingClassifier

In [68]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.externals.joblib import Parallel, delayed

In [69]:
from sklearn.metrics import average_precision_score, precision_recall_curve, f1_score, roc_curve, roc_auc_score

In [70]:
from sklearn.ensemble import RandomForestClassifier

In [71]:
from itertools import combinations
from sklearn.model_selection import GroupShuffleSplit

In [72]:
pd.read_csv('Ripple/K562_enhanceronly_features.txt', delimiter='\t').set_index('Pair').columns

Index(['K562_Ctcf_E', 'K562_Dnase1_E', 'K562_H3k27ac_E', 'K562_H3k27me3_E',
       'K562_H3k36me3_E', 'K562_H3k4me2_E', 'K562_H3k9ac_E', 'K562_H4k20me1_E',
       'K562_Rad21_E', 'K562_Tbp_E', 'K562_Ctcf_P', 'K562_Dnase1_P',
       'K562_H3k27ac_P', 'K562_H3k27me3_P', 'K562_H3k36me3_P',
       'K562_H3k4me2_P', 'K562_H3k9ac_P', 'K562_H4k20me1_P', 'K562_Rad21_P',
       'K562_Tbp_P', 'K562_Correlation', 'K562_Exp', 'Class'],
      dtype='object')

In [73]:
from sklearn.model_selection import cross_val_score, StratifiedKFold, KFold
from sklearn.ensemble import GradientBoostingClassifier
import numpy as np
from sklearn.metrics import average_precision_score, precision_recall_curve, f1_score

def aupr_auroc_f1(estimator, X, y):
    probs = estimator.predict_proba(X)
    preds = estimator.predict(X)
    aupr = average_precision_score(y, probs[:,1])
    auroc = roc_auc_score(y, probs[:,1])
    f1 = f1_score(y, preds)
    return aupr, auroc, f1

def train_a_ripple(train_data, train_label, test_data, test_label):
    rfc = RandomForestClassifier(**params)
    rfc.fit(train_data, train_label)
    return aupr_auroc_f1(rfc, test_data, test_label)

def train_an_ripple_intra(train_data, train_label, groups, seed):
    rfc = RandomForestClassifier(**params)
    train_data, train_label, groups = shuffle(train_data, train_label, groups, random_state=seed)
    cv = GroupKFold(5)
    cv = cv.split(train_data, train_label, groups)
    probs = cross_val_predict(rfc, train_data, train_label, cv=cv, n_jobs=1, method='predict_proba')
    preds = probs[:,1]>=0.5
    return (average_precision_score(train_label, probs[:,1]),
            roc_auc_score(train_label, probs[:,1]),
            f1_score(train_label, preds))

def train_ripple_cv(to_shuffle):
    train = pd.read_csv('Ripple/K562_enhanceronly_features.txt', delimiter='\t').set_index('Pair')
    
    chrom_info = {}
    keys = ['chrom1', 'start1', 'end1', 'chrom2', 'start2', 'end2']
    for k in keys:
        chrom_info[k] = []

    for i in train.index:
        chrom1, start1, end1 = i.split('-')[0].split('_')
        chrom2, start2, end2 = i.split('-')[1].split('_')
        
        for k,d in zip(keys, [chrom1, start1, end1, chrom2, start2, end2]):
            if d.isdigit():
                d = int(d)
            chrom_info[k].append(d)
        
    for k in keys:
        train[k] = chrom_info[k]
    train = train.sort_values(by=keys)
    train = train.drop(keys, axis=1)
    
    
    cv = StratifiedKFold(n_splits = 5, shuffle = to_shuffle, random_state=0)
    cv = cv.split(train.iloc[:, :-1], train['Class'])
    
    tasks = []
    for train_idx, test_idx in cv:
        tasks.append(delayed(train_a_ripple)(train.ix[train_idx, :-1], train.ix[train_idx, 'Class'],
                                        train.ix[test_idx, :-1], train.ix[test_idx, 'Class']))

    
    with Parallel(n_jobs=20) as parallel:
        scores = parallel(tasks)
    auprcs = [x[0] for x in scores]
    aurocs = [x[1] for x in scores]
    f1s = [x[2] for x in scores]

    print("\nCV")
    print(np.mean(auprcs), np.mean(aurocs), np.mean(f1s))
    print(np.std(auprcs), np.std(aurocs), np.std(f1s))
    print('dummy:', np.sum(train['Class'])/train.shape[0])
    


In [74]:
def train_ripple_intra():
    train = pd.read_csv('Ripple/K562_enhanceronly_features.txt', delimiter='\t').set_index('Pair')       
    train = train.sort_index()
    
    
    total_pos = np.sum(train['Class'])
    total_neg = np.sum(train['Class']==0)
    all_chrom_counts = {}
    print('counting')
    
    
    for i in train.index:
        curr_chrom = i.split('_')[0]+'_'
    
        if curr_chrom not in all_chrom_counts:
            all_chrom_counts[curr_chrom] = [0,0]
        if train.loc[i, 'Class'] == 0:
            all_chrom_counts[curr_chrom][1] += 1
        else:
            all_chrom_counts[curr_chrom][0] += 1
    all_chroms = sorted(list(all_chrom_counts.keys()))
    chrom_to_group_map = dict([(all_chroms[i], i) for i in range(len(all_chroms))])
    groups = []
    for i in train.index:
        curr_chrom = i.split('_')[0]+'_'
        groups.append(chrom_to_group_map[curr_chrom])
    tasks = []
    dummy_avg = []
    for seed in [0,29,192,92,123,2212]:

        tasks.append(delayed(train_an_jeme_intra)(train.ix[:,:-1], train['Class'], groups, seed))
        dummy_avg.append(np.sum(train['Class']) / len(train['Class']))
        
    
    scores = []
    with Parallel(n_jobs=20) as parallel:
        scores = parallel(tasks)
    auprcs = [x[0] for x in scores]
    aurocs = [x[1] for x in scores]
    f1s = [x[2] for x in scores]

    print("\nIntra")
    print(np.mean(auprcs), np.mean(aurocs), np.mean(f1s))
    print(np.std(auprcs), np.std(aurocs), np.std(f1s))
    print("dummpy", dummy_avg)

In [75]:
def train_ripple():
    train = pd.read_csv('Ripple/K562_enhanceronly_features.txt', delimiter='\t').set_index('Pair')  
    test = pd.read_csv('Ripple/Gm12878_enhanceronly_features.txt', delimiter='\t').set_index('Pair')  

    train = train.sort_index()
    test = test.sort_index()
    train_data = train.iloc[:, :-1]
    train_labels = train['Class']

    test_data = test.iloc[:, :-1]
    test_labels = test['Class']
    train_data.fillna(0)
    test_data.fillna(0)

    estimator = RandomForestClassifier(**params)

    estimator.fit(train_data, train_labels)
    probs = estimator.predict_proba(test_data)
    preds = estimator.predict(test_data)
    print("\nAcross sample")
    print(average_precision_score(test_labels, probs[:,1]), 
          roc_auc_score(test_labels, probs[:,1]),
          f1_score(test_labels, preds))
    print("dummpy", np.sum(test_labels) / len(test_labels))

In [76]:
for i in [5,10,None]:
    params = {'n_estimators': 100, 'max_depth':i,
              'random_state': 0, 'n_jobs': 10}
    #train_ripple_cv(True)
    #train_ripple_cv(False)
    train_ripple_intra()
    #train_ripple()

counting

Intra
0.643721751398 0.658038291799 0.565338953232
0.0018264136015 0.00132044377955 0.00457227088557
dummpy [0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
counting

Intra
0.625063511801 0.648328173817 0.555084430666
0.00497496007052 0.00391229775708 0.00942232703241
dummpy [0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
counting

Intra
0.591096661447 0.625029308044 0.557593031386
0.00410794892139 0.00428825948954 0.00531868479096
dummpy [0.5, 0.5, 0.5, 0.5, 0.5, 0.5]


In [77]:
from sklearn.model_selection import GroupKFold

In [78]:
from sklearn.utils import shuffle

In [79]:
def train_an_jeme(train_data, train_label, test_data, test_label):
    rfc = RandomForestClassifier(**params)
    rfc.fit(train_data, train_label)
    return aupr_auroc_f1(rfc, test_data, test_label)

In [80]:
def train_an_jeme_intra(train_data, train_label, groups, seed):
    rfc = RandomForestClassifier(**params)
    train_data, train_label, groups = shuffle(train_data, train_label, groups, random_state=seed)
    cv = GroupKFold(5)
    cv = cv.split(train_data, train_label, groups)
    probs = cross_val_predict(rfc, train_data, train_label, cv=cv, n_jobs=1, method='predict_proba')
    preds = probs[:,1]>=0.5
    return (average_precision_score(train_label, probs[:,1]),
            roc_auc_score(train_label, probs[:,1]),
            f1_score(train_label, preds))

In [81]:
def train_jeme_cv(columns, to_shuffle):
    train = pd.read_csv('./JEME/K562/all.sw.se.sp.csv').set_index('id3')
    train = train.drop(['id1', 'id2'], axis=1)
    
    #train['id4'] = np.log10(train['id4'])
    chrom_info = {}
    keys = ['chrom1', 'start1', 'end1', 'chrom2', 'start2', 'end2']
    for k in keys:
        chrom_info[k] = []
    for idx in train.index:
        idx = idx.replace(':', ' ').replace('-', ' ').replace('_', ' ').replace('$', ' ').strip()
        tokens = idx.split()
        for k, d in zip(keys, tokens):
            if d.isdigit():
                d = int(d)
            chrom_info[k].append(d)

    for k in keys:
        train[k] = chrom_info[k]
    train = train.sort_values(by=keys)
    train = train.drop(keys, axis=1)
    cv = StratifiedKFold(n_splits = 5, shuffle=to_shuffle, random_state = 0)
    cv = cv.split(train.loc[:, columns], train['id21'])

    
    tasks = []
    for train_idx, test_idx in cv:
        tasks.append(delayed(train_an_jeme)(train.ix[train_idx, columns], train.ix[train_idx, 'id21'],
                                        train.ix[test_idx, columns], train.ix[test_idx, 'id21']))
    
    with Parallel(n_jobs=20) as parallel:
        scores = parallel(tasks)
    auprcs = [x[0] for x in scores]
    aurocs = [x[1] for x in scores]
    f1s = [x[2] for x in scores]

    print("\nCV")
    print(np.mean(auprcs), np.mean(aurocs), np.mean(f1s))
    print(np.std(auprcs), np.std(aurocs), np.std(f1s))
    print('dummy:', np.sum(train['id21'])/train.shape[0])



In [82]:
def train_jeme_intra(columns):
    train = pd.read_csv('./JEME/K562/all.sw.se.sp.csv').set_index('id3')
    train = train.drop(['id1', 'id2'], axis=1)
    #train['id4'] = np.log10(train['id4'])
    
    total_pos = np.sum(train['id21'])
    total_neg = np.sum(train['id21']==0)
    all_chrom_counts = {}
    print('counting')
    for i in train.index:
        curr_chrom = i.split(':')[0]+':'
    
        if curr_chrom not in all_chrom_counts:
            all_chrom_counts[curr_chrom] = [0,0]
        if train.loc[i, 'id21'] == 0:
            all_chrom_counts[curr_chrom][1] += 1
        else:
            all_chrom_counts[curr_chrom][0] += 1
    all_chroms = sorted(list(all_chrom_counts.keys()))
    chrom_to_group_map = dict([(all_chroms[i], i) for i in range(len(all_chroms))])
    groups = []
    for i in train.index:
        curr_chrom = i.split(':')[0]+':'
        groups.append(chrom_to_group_map[curr_chrom])
    tasks = []
    dummy_avg = []
    for seed in range(0,200,10):

        tasks.append(delayed(train_an_jeme_intra)(train.ix[:,columns], train['id21'], groups, seed))
        dummy_avg.append(np.sum(train['id21']) / len(train['id21']))
        
    scores = []
    with Parallel(n_jobs=20) as parallel:
        scores = parallel(tasks)
    auprcs = [x[0] for x in scores]
    aurocs = [x[1] for x in scores]
    f1s = [x[2] for x in scores]
    print("\nIntra")
    print(np.mean(auprcs), np.mean(aurocs), np.mean(f1s))
    print(np.std(auprcs), np.std(aurocs), np.std(f1s))
    print("dummpy", dummy_avg)
    
    
    
def train_jeme_intra_cv(columns):
    train = pd.read_csv('./JEME/K562/all.sw.se.sp.csv').set_index('id3')
    train = train.drop(['id1', 'id2'], axis=1)
    #train['id4'] = np.log10(train['id4'])
    
    total_pos = np.sum(train['id21'])
    total_neg = np.sum(train['id21']==0)
    all_chrom_counts = {}
    print('counting')
    for i in train.index:
        curr_chrom = i.split(':')[0]+':'
    
        if curr_chrom not in all_chrom_counts:
            all_chrom_counts[curr_chrom] = [0,0]
        if train.loc[i, 'id21'] == 0:
            all_chrom_counts[curr_chrom][1] += 1
        else:
            all_chrom_counts[curr_chrom][0] += 1
    all_chroms = sorted(list(all_chrom_counts.keys()))
    chrom_to_group_map = dict([(all_chroms[i], i) for i in range(len(all_chroms))])
    groups = []
    for i in train.index:
        curr_chrom = i.split(':')[0]+':'
        groups.append(chrom_to_group_map[curr_chrom])
    tasks = []
    dummy_avg = []
    cv = GroupKFold(n_splits = 5)
    cv = cv.split(train.loc[:, columns], train['id21'], groups)

    
    tasks = []
    for train_idx, test_idx in cv:
        tasks.append(delayed(train_an_jeme)(train.ix[train_idx, columns], train.ix[train_idx, 'id21'],
                                        train.ix[test_idx, columns], train.ix[test_idx, 'id21']))
        
    scores = []
    with Parallel(n_jobs=20) as parallel:
        scores = parallel(tasks)
    auprcs = [x[0] for x in scores]
    aurocs = [x[1] for x in scores]
    f1s = [x[2] for x in scores]
    print("\nIntra")
    print(np.mean(auprcs), np.mean(aurocs), np.mean(f1s))
    print(np.std(auprcs), np.std(aurocs), np.std(f1s))
    print("dummpy", dummy_avg)
    

In [83]:
def train_jeme(columns):
    train = pd.read_csv('./JEME/K562/all.sw.se.sp.csv').set_index('id3')
    train = train.drop(['id1', 'id2'], axis=1)

    #train['id4'] = np.log10(train['id4'])
    test = pd.read_csv('./JEME/GM12878/all.sw.se.sp.csv').set_index('id3')
    test = test.drop(['id1', 'id2'], axis=1)

    
    train_data = train.loc[:, columns]

    test_data = test.loc[:, columns]
    
    train_labels = train['id21']

    test_labels = test['id21']
    
    rfc = RandomForestClassifier(**params)
    rfc.fit(train_data, train_labels)
    probs = rfc.predict_proba(test_data)
    preds = rfc.predict(test_data)
    print("\n\nAcross sample")
    print(average_precision_score(test_labels, probs[:,1]), 
          roc_auc_score(test_labels, probs[:,1]),
          f1_score(test_labels, preds))
    print("dummpy", np.sum(test_labels) / len(test_labels))

In [84]:
for i in [5,10,None]:
    columns = ['id%d' % i for i in range(4,21)]
    params = {'n_estimators': 100, 'max_depth': i,
              'random_state': 0, 'n_jobs': 10}
    #train_jeme_cv(columns, to_shuffle=True)
    #train_jeme_cv(columns, to_shuffle=False)
    train_jeme_intra(columns)
    #train_jeme_intra_cv(columns)
    #train_jeme(columns)

counting

Intra
0.548928446424 0.559509783144 0.494832395946
0.00412793790944 0.00445781997055 0.00931789301525
dummpy [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
counting

Intra
0.550598853542 0.556610627086 0.459737685022
0.00478817411924 0.00523194890354 0.0101790753056
dummpy [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
counting

Intra
0.553426045043 0.55804617951 0.446072084234
0.00478764981287 0.00477943051507 0.00833452237395
dummpy [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]


In [85]:
for i in [5,10,None]:
    columns = ['id4']
    params = {'n_estimators': 100, 'max_depth': i,
              'random_state': 0, 'n_jobs': 10}
    #train_jeme_cv(columns, to_shuffle=True)
    #train_jeme_cv(columns, to_shuffle=False)
    train_jeme_intra(columns)
    #train_jeme(columns)

counting

Intra
0.404937078308 0.342110686244 0.355817061383
0.00215615528353 0.00261283193717 0.00693806016824
dummpy [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
counting

Intra
0.442220618997 0.404611417591 0.431517381963
0.00168411012339 0.00236190096386 0.00673259741878
dummpy [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
counting

Intra
0.494017020984 0.490500878266 0.503843786089
0.00283754620843 0.0022114755283 0.000358154332886
dummpy [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]


In [86]:
for i in [5,10,None]:
    columns = ['id%d' % i for i in range(5,21)]
    params = {'n_estimators': 100, 'max_depth':i,
              'random_state': 0, 'n_jobs': 10}
    #train_jeme_cv(columns, to_shuffle=True)
    #train_jeme_cv(columns, to_shuffle=False)
    train_jeme_intra(columns)
    #train_jeme(columns)

counting

Intra
0.549380467654 0.560359380546 0.499226638026
0.00441518253233 0.00378483182137 0.00984107857963
dummpy [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
counting

Intra
0.554124528342 0.561291896418 0.472930501216
0.00724440785302 0.00681383846039 0.0104196220015
dummpy [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
counting

Intra
0.552105678785 0.560672949531 0.449255842202
0.00565355224459 0.00689092930662 0.00869430860741
dummpy [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]


In [87]:
columns = ['id%d' % i for i in range(9,21)]
for i in [5,10,None]:
    print(i)
    params = {'n_estimators': 100, 'max_depth': i,
              'random_state': 0, 'n_jobs': 10}
    #train_jeme_cv(columns, to_shuffle=True)
    #train_jeme_cv(columns, to_shuffle=False)
    train_jeme_intra(columns)
    #train_jeme(columns)
    print('\n')

5
counting

Intra
0.548622864868 0.55710937307 0.50315297499
0.00480894594456 0.00467342765902 0.00787009937873
dummpy [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]


10
counting

Intra
0.547791507673 0.548393182418 0.46409804488
0.00468091972693 0.00430869287688 0.00854695431817
dummpy [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]


None
counting

Intra
0.541394467523 0.544461364739 0.441148298451
0.00436242492478 0.00609369636208 0.010607436905
dummpy [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]




In [88]:
columns = ['id%d' % i for i in range(9,21)] + ['id4']
for i in [5,10,None]:
    print(i)
    params = {'n_estimators': 100, 'max_depth': i,
              'random_state': 0, 'n_jobs': 10}
    #train_jeme_cv(columns, to_shuffle=True)
    #train_jeme_cv(columns, to_shuffle=False)
    train_jeme_intra(columns)
    #train_jeme(columns)
    print('\n')

5
counting

Intra
0.548897288635 0.558298282863 0.503351943028
0.00500178569337 0.00469266062055 0.00815400051882
dummpy [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]


10
counting

Intra
0.547798269426 0.547703311148 0.460300182332
0.00440201160898 0.00606121618874 0.00847391144899
dummpy [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]


None
counting

Intra
0.543362046592 0.543578027873 0.440051218437
0.00697200633646 0.00726172594965 0.0093805819978
dummpy [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]


